In [17]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames, Statistics, MLJ, Plots

In [12]:
### synthtic points

X, yy = make_blobs(50, 2; centers=2, cluster_std=[1.0, 3.0])
points = Matrix(DataFrame(X));

In [13]:
# J = n of points
# I clusters
# D = dimension
# points[j,d]

J = 50;
I = 2;
D = 2;

In [14]:
model = JuMP.Model(Gurobi.Optimizer);

@variable(model, gamma[1:J]);
@variable(model, z[1:J, 1:I],Bin);
@variable(model, mu[1:J, 1:I] >=0);
@variable(model, r[1:J, 1:I] >=0);
## for dimension of data
@variable(model, x[1:I, 1:D] >=0);
@variable(model, y[1:J, 1:D, 1:I] >=0);

### one point per cluster, one cluster per point
#@constraint(model, [i = 1:I], sum(z[:,i]) == 1);
@constraint(model, [j = 1:J], sum(z[j,:]) == 1);

@constraint(model, [i=1:I, j=1:J, d=1:D], sum(y[j,d,i] for d=1:D) == r[j,i]);
@constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= x[i,d] - points[j,d]);
@constraint(model, [i=1:I, j=1:J, d=1:D], y[j,d,i] >= -(x[i,d] - points[j,d]));

@constraint(model, [i=1:I, j=1:J], gamma[j]>= r[j,i]-mu[j,i]);

@constraint(model, [i = 1:I, j = 1:J], 1000(1-z[j,i]) >= mu[j,i]);

@objective(model, Min, sum(gamma[j] for j=1:J));

optimize!(model);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-04
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 850 rows, 554 columns and 2000 nonzeros
Model fingerprint: 0x0b736ed8
Variable types: 454 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e-01, 1e+03]
Presolve removed 462 rows and 306 columns
Presolve time: 0.00s
Presolved: 388 rows, 248 columns, 976 nonzeros
Variable types: 198 continuous, 50 integer (50 binary)
Found heuristic solution: objective 540.4370280

Root relaxation: objective 1.522350e+02, 257 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0    

In [9]:
value.(z)

50×2 Matrix{Float64}:
  1.0  0.0
  1.0  0.0
  1.0  0.0
  1.0  0.0
 -0.0  1.0
  1.0  0.0
 -0.0  1.0
  1.0  0.0
 -0.0  1.0
  1.0  0.0
  1.0  0.0
 -0.0  1.0
 -0.0  1.0
  ⋮    
  1.0  0.0
 -0.0  1.0
  1.0  0.0
  1.0  0.0
  1.0  0.0
 -0.0  1.0
  1.0  0.0
  1.0  0.0
 -0.0  1.0
  1.0  0.0
  1.0  0.0
 -0.0  1.0

In [21]:
value.(x)

2×2 Matrix{Float64}:
 7.85903   0.0
 4.34395  10.1691